### Read and set the data into list

In [2]:
import platypus
dir(platypus)

['ABCMeta',
 'AbstractGeneticAlgorithm',
 'AdaptiveGridArchive',
 'AdaptiveTimeContinuation',
 'Algorithm',
 'ApplyEvaluator',
 'Archive',
 'AttributeDominance',
 'Binary',
 'BitFlip',
 'CF1',
 'CF10',
 'CF2',
 'CF3',
 'CF4',
 'CF5',
 'CF6',
 'CF7',
 'CF8',
 'CF9',
 'CMAES',
 'CompoundMutation',
 'CompoundOperator',
 'Constraint',
 'DTLZ1',
 'DTLZ2',
 'DTLZ3',
 'DTLZ4',
 'DTLZ7',
 'DifferentialEvolution',
 'DistanceMatrix',
 'Dominance',
 'EPSILON',
 'EpsMOEA',
 'EpsNSGAII',
 'EpsilonBoxArchive',
 'EpsilonDominance',
 'EpsilonIndicator',
 'EpsilonProgressContinuation',
 'Evaluator',
 'EvolutionaryStrategy',
 'ExperimentJob',
 'FitnessArchive',
 'FitnessEvaluator',
 'FixedLengthArray',
 'GAOperator',
 'GDE3',
 'GenerationalDistance',
 'Generator',
 'GeneticAlgorithm',
 'HUX',
 'Hypervolume',
 'HypervolumeFitnessEvaluator',
 'IBEA',
 'Indicator',
 'IndicatorJob',
 'InjectedPopulation',
 'Insertion',
 'Integer',
 'InvertedGenerationalDistance',
 'Job',
 'LOGGER',
 'MOEAD',
 'MapEvaluator'

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

inputFileName = "Data_Knapsack.xlsx"
param_data = pd.read_excel(inputFileName, "Param", skiprows=0)
data = pd.read_excel(inputFileName, "Instance", skiprows=0)


In [64]:
nbItems = 1000 #param_data['Value'][0]  #10000

items = data['Item'].truncate(after=nbItems-1)
weights = data['Weight'].truncate(after=nbItems-1)
values = data['Value'].truncate(after=nbItems-1)

    
totalWeight = sum(weights[iItem] for iItem in range(nbItems))
capacity = 0.5 * totalWeight

### Genetic algorithm

In [65]:
def computeFitness(population):  #population is a np.array with N=populationSize rows, and M=nbItems columns
    
    populationSize = population.shape[0]
    results = np.empty(populationSize)
    
    for i in range(populationSize):
        weight_i = np.dot(population[i,:], weights)
        value_i  = np.dot(population[i,:], values)
        if weight_i<=capacity :
            results[i] = value_i
        else:
            results[i] = 0
        
    return results #return a np.array with N=populationSize rows, and M=1 columns

In [66]:
def select(population, fitnessResults, nbParentsSelected):
    fitnessCopy = computeFitness(population)
    parents = np.empty((nbParentsSelected, population.shape[1]))
    for i in range(nbParentsSelected):
        max_fitness_idx=np.where(fitnessCopy==np.max(fitnessCopy))
        parents[i,:]=population[max_fitness_idx[0][0],:]
        fitnessCopy[max_fitness_idx[0][0]]=-99999
    
    return parents

In [67]:
def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size)
    nbParents = parents.shape[0]
 
    for k in range(offspring_size[0]):
        
        # The point at which crossover takes place between two parents
        crossover_point = int(np.random.rand() * offspring_size[1])
        
        # Index of the first and second parent to mate
        parent1_id = np.random.randint(nbParents)
        parent2_id = np.random.randint(nbParents)
        if (parent2_id == parent1_id) : parent2_id += 1
        parent1_id = parent1_id%nbParents  #use modulo to make it < nbParents
        parent2_id = parent2_id%nbParents  #use modulo to make it < nbParents
        
        
        # The new offspring will have its 1st half of its genes from the 1st parent and its 2nd half of its genes from the 2nd parent
        offspring[k, 0:crossover_point] = parents[parent1_id, 0:crossover_point]
        offspring[k, crossover_point:] = parents[parent2_id, crossover_point:]
        
    return offspring

In [68]:
def mutation(offspring_crossover, mutationProbability):
    
    nbIndividuals = offspring_crossover.shape[0]
    
    for i in range(nbIndividuals):
        rd=np.random.rand()
        proceedToMutation = rd>mutationProbability
        if proceedToMutation:
            index=int(np.random.rand() * offspring_crossover.shape[1])
            if offspring_crossover[i,index]==0 :
                offspring_crossover[i,index]==1
            else:
                offspring_crossover[i,index]==0
        
    return offspring_crossover

In [70]:
#Define parameters
populationSize = 2000
nbGenerations = 100
nbParentsSelected = int(0.2 * populationSize)  #number of parents you will select at each generation
mutationProbability = 0.5  #each crossover has this probability to mute 


#Initialisation
population = np.random.randint(2, size=(populationSize, nbItems))

for iGeneration in range(nbGenerations):
    
    # Measuring the fitness of each chromosome in the population
    fitnessResults = computeFitness(population)

    # Selecting the best parents in the population for mating
    parents = select(population, fitnessResults, nbParentsSelected)

    # Generating next generation using crossover
    offspring_crossover = crossover(parents, offspring_size=(populationSize-nbParentsSelected, nbItems))

    # Adding some variations to the offsrping using mutation
    offspring_mutation = mutation(offspring_crossover, mutationProbability)

    # Creating the new population based on the parents and offspring
    population[0:parents.shape[0], :] = parents
    population[parents.shape[0]:, :] = offspring_mutation

    # The best result in the current iteration.
    print("Best result : ", np.max(computeFitness(population)))

    
#Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
fitness = computeFitness(population)
# Then return the index of that solution corresponding to the best fitness.
best_match_idx = np.where(fitness == np.max(fitness))

print("Best solution : ", population[best_match_idx[0][0], :])
print("Best solution fitness : ", fitness[best_match_idx[0][0]])

Best result :  102196.0
Best result :  102646.0
Best result :  102683.0
Best result :  103035.0
Best result :  103531.0
Best result :  103614.0
Best result :  103674.0
Best result :  103688.0
Best result :  104054.0
Best result :  104221.0
Best result :  104616.0
Best result :  104706.0
Best result :  104846.0
Best result :  105105.0
Best result :  105118.0
Best result :  105161.0
Best result :  105382.0
Best result :  105564.0
Best result :  105564.0
Best result :  105564.0
Best result :  105767.0
Best result :  105875.0
Best result :  105948.0
Best result :  106008.0
Best result :  106122.0
Best result :  106136.0
Best result :  106136.0
Best result :  106245.0
Best result :  106245.0
Best result :  106252.0
Best result :  106252.0
Best result :  106252.0
Best result :  106283.0
Best result :  106283.0
Best result :  106283.0
Best result :  106283.0
Best result :  106313.0
Best result :  106313.0
Best result :  106313.0
Best result :  106326.0
Best result :  106342.0
Best result :  1